In [1]:
import gym
import torch
from torch import nn
from collections import namedtuple, deque
import itertools
from copy import deepcopy
import random
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import tqdm

/home/devjac/Code/python/learn-pytorch/.venv/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


In [2]:
RENDER = False

In [3]:
ALPHA = 0.1  # How much we value entropy / exploration, increasing this will increase exploration.
GAMMA = 0.99  # How much we value future rewards.
TAU = 0.001  # How much q_target is updated when polyak averaging (step 15).
POLICY_LR = 0.001  # Policy learning rate.
Q_LR = 0.001  # Q learning rate.

In [4]:
env = gym.make("CartPole-v1")

In [5]:
SARS = namedtuple('SARS', 'state, action, reward, next_state, t, failed, limit')

In [6]:
softmax = nn.Softmax(dim=0)
input = torch.tensor([1, 2, 3], dtype=float)
display(input)
output = softmax(input)
display(output)
sum(output)

tensor([1., 2., 3.], dtype=torch.float64)

tensor([0.0900, 0.2447, 0.6652], dtype=torch.float64)

tensor(1.0000, dtype=torch.float64)

In [7]:
softmax = nn.Softmax(dim=1)
input = torch.tensor([[1, 2, 3], [1, 2, 3], [3, 3, 3]], dtype=float)
display(input)
output = softmax(input)
display(output)
sum(output)

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [3., 3., 3.]], dtype=torch.float64)

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652],
        [0.3333, 0.3333, 0.3333]], dtype=torch.float64)

tensor([0.5134, 0.8228, 1.6638], dtype=torch.float64)

In [8]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 600),
            nn.ReLU(),
            nn.Linear(600, 200),
            nn.ReLU(),
            nn.Linear(200, output_size)
        )

    def forward(self, x):
        nn_out = self.linear_relu_stack(x)
        return nn.Softmax(dim=1)(nn_out)

    def __call__(self, x):
        raise RuntimeError("Use forward")

In [9]:
policy_network = PolicyNetwork(4, 2)
policy_network

PolicyNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=2, bias=True)
  )
)

In [10]:
torch.cuda.is_available()

True

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
mock_states = torch.rand(5, 4)
mock_states

tensor([[0.5955, 0.5279, 0.0574, 0.9862],
        [0.7579, 0.5896, 0.7630, 0.6750],
        [0.2088, 0.8976, 0.4527, 0.6077],
        [0.0098, 0.2516, 0.1567, 0.8262],
        [0.2279, 0.5343, 0.1252, 0.8394]])

In [13]:
policy_network.forward(mock_states)

tensor([[0.5378, 0.4622],
        [0.5489, 0.4511],
        [0.5363, 0.4637],
        [0.5313, 0.4687],
        [0.5301, 0.4699]], grad_fn=<SoftmaxBackward0>)

In [14]:
class QNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 600),
            nn.ReLU(),
            nn.Linear(600, 200),
            nn.ReLU(),
            nn.Linear(200, output_size)
        )

    def forward(self, x):
        nn_out = self.linear_relu_stack(x)
        return nn_out
    
    def __call__(self, x):
        raise RuntimeError("Use forward")

In [15]:
q_network = QNetwork(4, 2)
q_network

QNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=2, bias=True)
  )
)

In [16]:
q_network.forward(mock_states)

tensor([[0.0071, 0.0846],
        [0.0388, 0.0882],
        [0.0511, 0.0748],
        [0.0644, 0.0569],
        [0.0388, 0.0773]], grad_fn=<AddmmBackward0>)

In [17]:
def run_episode(action_f, step_f, env, policy):
    episode_reward = 0
    s = env.reset()
    for t in itertools.count(start=0):
        a = action_f(policy, s)
        next_state, reward, failed, info = env.step(a)
        episode_reward += reward
        assert t <= env._max_episode_steps
        limit = t == env._max_episode_steps
        if limit:
            failed = False
        assert not (limit and failed)
        step_f(s, a, reward, next_state, t, failed, limit)
        if failed or limit:
            break
        s = next_state
    return episode_reward

In [18]:
class Policy:
    def __init__(self, env_state_size, env_action_space_size):
        self.policy_network = PolicyNetwork(env_state_size, env_action_space_size)
        self.q1_network = QNetwork(env_state_size, env_action_space_size)
        self.q2_network = QNetwork(env_state_size, env_action_space_size)
        self.q1_target_network = deepcopy(self.q1_network)
        self.q2_target_network = deepcopy(self.q2_network)
        self.policy_optimizer = torch.optim.SGD(self.policy_network.parameters(), lr=POLICY_LR)
        self.q1_optimizer = torch.optim.SGD(self.q1_network.parameters(), lr=Q_LR)
        self.q2_optimizer = torch.optim.SGD(self.q2_network.parameters(), lr=Q_LR)

In [19]:
replay_buffer = deque(maxlen=10_000)

In [20]:
env.action_space.n

2

In [21]:
env.observation_space.shape

(4,)

In [22]:
oss = env.observation_space.shape
if len(oss) != 1:
    raise RuntimeError(f'Unknown observation_space.shape: {oss}')
os_len = oss[0]
policy = Policy(os_len, env.action_space.n)

In [23]:
s = env.reset()
s

array([ 0.04659428, -0.02196541,  0.01238199, -0.04668314], dtype=float32)

In [24]:
s = torch.tensor(s).reshape((1, -1))
s

tensor([[ 0.0466, -0.0220,  0.0124, -0.0467]])

In [25]:
policy_output = policy.policy_network.forward(s)
policy_output

tensor([[0.4946, 0.5054]], grad_fn=<SoftmaxBackward0>)

In [26]:
action_weights = policy_output.reshape((-1,)).tolist()

In [27]:
list(range(4))

[0, 1, 2, 3]

In [28]:
action = random.choices(range(len(action_weights)), weights=action_weights)[0]
action

1

In [29]:
def action(policy, s):
    tensor_s = torch.tensor(s).reshape((1, -1))
    action_weights = policy.policy_network.forward(tensor_s).reshape((-1,)).tolist()
    action = random.choices(range(len(action_weights)), weights=action_weights)[0]
    return action

def step(initial_s, a, r, next_s, t, failed, limit):
    replay_buffer.append(SARS(initial_s, a, r, next_s, t, failed, limit))
    if RENDER:
        env.render()

In [30]:
run_episode(action, step, env, policy)

90.0

In [31]:
len(replay_buffer)

90

In [32]:
replay_buffer

deque([SARS(state=array([ 0.01705702, -0.00205287, -0.03202293,  0.00623765], dtype=float32), action=0, reward=1.0, next_state=array([ 0.01701596, -0.19670129, -0.03189818,  0.28864744], dtype=float32), t=0, failed=False, limit=False),
       SARS(state=array([ 0.01701596, -0.19670129, -0.03189818,  0.28864744], dtype=float32), action=1, reward=1.0, next_state=array([ 0.01308193, -0.00113932, -0.02612523, -0.01392275], dtype=float32), t=1, failed=False, limit=False),
       SARS(state=array([ 0.01308193, -0.00113932, -0.02612523, -0.01392275], dtype=float32), action=0, reward=1.0, next_state=array([ 0.01305915, -0.19587706, -0.02640369,  0.27040422], dtype=float32), t=2, failed=False, limit=False),
       SARS(state=array([ 0.01305915, -0.19587706, -0.02640369,  0.27040422], dtype=float32), action=0, reward=1.0, next_state=array([ 0.00914161, -0.39061245, -0.0209956 ,  0.55464375], dtype=float32), t=3, failed=False, limit=False),
       SARS(state=array([ 0.00914161, -0.39061245, -0.02

# Polyak Averaging

In [33]:
test_parameter_1 = next(policy.policy_network.named_parameters())[1]
test_parameter_1

Parameter containing:
tensor([[-0.4083,  0.0696, -0.2966, -0.0051],
        [ 0.0843,  0.1779, -0.2049,  0.4490],
        [ 0.0683, -0.1412, -0.4319, -0.0770],
        ...,
        [-0.0125, -0.0770,  0.1609, -0.4340],
        [-0.0757,  0.0085,  0.4197, -0.1831],
        [-0.3773, -0.4425,  0.2319,  0.0120]], requires_grad=True)

In [34]:
test_parameter_2 = test_parameter_1 * 0 + 0.0128
test_parameter_2

tensor([[0.0128, 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128, 0.0128],
        ...,
        [0.0128, 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128, 0.0128]], grad_fn=<AddBackward0>)

In [35]:
test_parameter_1 * 0.9 + test_parameter_2 * 0.1

tensor([[-0.3662,  0.0639, -0.2657, -0.0033],
        [ 0.0772,  0.1614, -0.1831,  0.4054],
        [ 0.0627, -0.1258, -0.3874, -0.0680],
        ...,
        [-0.0100, -0.0680,  0.1461, -0.3893],
        [-0.0669,  0.0090,  0.3790, -0.1635],
        [-0.3382, -0.3969,  0.2100,  0.0121]], grad_fn=<AddBackward0>)

In [36]:
def polyak_update(network_to_update, target_network, tau=0.001):
    with torch.no_grad():
        for to_update, target in zip(network_to_update.parameters(), target_network.parameters()):
            to_update *= 1-tau
            to_update += target * tau

In [37]:
test_network_1 = QNetwork(5, 3)
test_network_2 = QNetwork(5, 3)
display(list(test_network_1.parameters())[0])
display(list(test_network_2.parameters())[0])
polyak_update(test_network_2, test_network_1, 0.1)
display(list(test_network_1.parameters())[0])
display(list(test_network_2.parameters())[0])

Parameter containing:
tensor([[-0.4177, -0.2698,  0.1713, -0.4210, -0.0153],
        [ 0.0030,  0.2139,  0.3215, -0.0469, -0.0017],
        [ 0.4171,  0.0285,  0.3802, -0.0684,  0.0761],
        ...,
        [-0.4030, -0.0351, -0.2655, -0.3643,  0.0377],
        [ 0.2405, -0.3039,  0.0783,  0.3646,  0.0467],
        [-0.2339,  0.0512, -0.1346, -0.3551, -0.3111]], requires_grad=True)

Parameter containing:
tensor([[-0.3832, -0.1674, -0.2559, -0.2801,  0.0834],
        [ 0.1957,  0.1690,  0.0934, -0.2222, -0.3882],
        [ 0.4304, -0.0352,  0.2414,  0.0199, -0.4374],
        ...,
        [-0.4034, -0.2170, -0.3155,  0.0643, -0.1919],
        [ 0.2143,  0.2310, -0.1604,  0.3832,  0.0863],
        [-0.3501, -0.1050,  0.0482, -0.3175, -0.2855]], requires_grad=True)

Parameter containing:
tensor([[-0.4177, -0.2698,  0.1713, -0.4210, -0.0153],
        [ 0.0030,  0.2139,  0.3215, -0.0469, -0.0017],
        [ 0.4171,  0.0285,  0.3802, -0.0684,  0.0761],
        ...,
        [-0.4030, -0.0351, -0.2655, -0.3643,  0.0377],
        [ 0.2405, -0.3039,  0.0783,  0.3646,  0.0467],
        [-0.2339,  0.0512, -0.1346, -0.3551, -0.3111]], requires_grad=True)

Parameter containing:
tensor([[-0.3867, -0.1776, -0.2132, -0.2942,  0.0735],
        [ 0.1764,  0.1735,  0.1162, -0.2047, -0.3495],
        [ 0.4291, -0.0288,  0.2553,  0.0110, -0.3861],
        ...,
        [-0.4034, -0.1988, -0.3105,  0.0215, -0.1690],
        [ 0.2169,  0.1775, -0.1365,  0.3814,  0.0823],
        [-0.3385, -0.0893,  0.0299, -0.3212, -0.2881]], requires_grad=True)

# Log Experiments

In [38]:
for p in [0.99, 0.9, 0.8, 0.6, 0.5]:
    logs = -np.log([p, 1-p])
    display(p, logs, sum(logs))

0.99

array([0.01005034, 4.60517019])

4.615220521841592

0.9

array([0.10536052, 2.30258509])

...

array([0.51082562, 0.91629073])

1.4271163556401456

0.5

array([0.69314718, 0.69314718])

1.3862943611198906

In [39]:
random.sample(replay_buffer, k=4)

[SARS(state=array([-0.02486775, -0.01806459,  0.07174376,  0.36024594], dtype=float32), action=0, reward=1.0, next_state=array([-0.02522904, -0.21412914,  0.07894868,  0.6746608 ], dtype=float32), t=24, failed=False, limit=False),
 SARS(state=array([-0.03470306, -0.21943721,  0.12453781,  0.7954223 ], dtype=float32), action=1, reward=1.0, next_state=array([-0.0390918 , -0.02622436,  0.14044626,  0.54436713], dtype=float32), t=29, failed=False, limit=False),
 SARS(state=array([ 0.01308193, -0.00113932, -0.02612523, -0.01392275], dtype=float32), action=0, reward=1.0, next_state=array([ 0.01305915, -0.19587706, -0.02640369,  0.27040422], dtype=float32), t=2, failed=False, limit=False),
 SARS(state=array([ 0.14721562,  1.3068826 ,  0.11028562, -0.79290545], dtype=float32), action=0, reward=1.0, next_state=array([ 0.17335327,  1.1104333 ,  0.09442751, -0.46766365], dtype=float32), t=45, failed=False, limit=False)]

# Training

![Psudocode](sac_psudocode.png)

Source: https://spinningup.openai.com/en/latest/algorithms/sac.html#pseudocode

In [40]:
def q_min(q1, q2, states, actions):
    
    def f(q):
        state_values = q.forward(states).detach()
        chosen_action_values = torch.sum(state_values * nn.functional.one_hot(actions), 1)
        return chosen_action_values
        
    return torch.minimum(*map(f, (q1, q2)))

In [41]:
    stats = {}
    # Step 11
    training_batch = random.sample(replay_buffer, k=min(len(replay_buffer), 100))
    # Prep
    states = torch.tensor(np.array([sars.state for sars in training_batch]), requires_grad=False)
    actions = torch.tensor(np.array([sars.action for sars in training_batch]), requires_grad=False)
    actions_hot = nn.functional.one_hot(actions)
    rewards = torch.tensor(np.array([sars.reward for sars in training_batch]), requires_grad=False)
    next_states = torch.tensor(np.array([sars.next_state for sars in training_batch]), requires_grad=False)
    fails = torch.tensor(np.array([sars.failed for sars in training_batch]), dtype=int, requires_grad=False)
    # Step 12
    next_action_probs = policy.policy_network.forward(next_states).detach()
    assert not next_action_probs.requires_grad
    next_actions_categorical = torch.distributions.Categorical(next_action_probs)
    next_actions = next_actions_categorical.sample()
    assert not next_actions.requires_grad
    next_actions_q_min = q_min(
        policy.q1_target_network,
        policy.q2_target_network,
        next_states,
        next_actions)
    assert not next_actions_q_min.requires_grad
    next_actions_log_probs = next_actions_categorical.log_prob(next_actions)
    assert not next_actions_log_probs.requires_grad
    y = rewards + GAMMA * (1-fails) * (next_actions_q_min - ALPHA * next_actions_log_probs)
    assert not y.requires_grad
    # Step 13
    for qi, q, opt in ((1, policy.q1_network, policy.q1_optimizer),
                       (2, policy.q2_network, policy.q2_optimizer)):
        assert not states.requires_grad
        assert not actions_hot.requires_grad
        q_state_action = torch.sum(q.forward(states) * actions_hot, 1)
        assert q_state_action.requires_grad
        q_loss = torch.mean((q_state_action - y)**2)
        stats[f'train/q_loss_{qi}'] = q_loss
        assert q_loss.requires_grad
        opt.zero_grad()
        q_loss.backward()
        opt.step()
    # Step 14
    action_probs = policy.policy_network.forward(states)
    assert action_probs.requires_grad
    actions_categorical = torch.distributions.Categorical(action_probs)
    actions = actions_categorical.sample()
    assert not actions.requires_grad
    actions_q_min = q_min(
        policy.q1_network,
        policy.q2_network,
        states,
        actions)
    assert not actions_q_min.requires_grad
    actions_log_probs = actions_categorical.log_prob(actions)
    assert actions_log_probs.requires_grad
    policy_loss = -1 * torch.mean(actions_q_min - ALPHA * actions_log_probs)
    stats['train/policy_loss'] = policy_loss
    policy.policy_optimizer.zero_grad()
    policy_loss.backward()
    policy.policy_optimizer.step()
    # Step 15
    polyak_update(policy.q1_target_network, policy.q1_network, tau=TAU)
    polyak_update(policy.q2_target_network, policy.q2_network, tau=TAU)
    stats

{'train/q_loss_1': tensor(1.0747, dtype=torch.float64, grad_fn=<MeanBackward0>),
 'train/q_loss_2': tensor(1.0940, dtype=torch.float64, grad_fn=<MeanBackward0>),
 'train/policy_loss': tensor(-0.0517, grad_fn=<MulBackward0>)}

In [42]:
def train(policy, replay_buffer):
    stats = {}
    # Step 11
    training_batch = random.sample(replay_buffer, k=min(len(replay_buffer), 100))
    # Prep
    states = torch.tensor(np.array([sars.state for sars in training_batch]), requires_grad=False)
    actions = torch.tensor(np.array([sars.action for sars in training_batch]), requires_grad=False)
    actions_hot = nn.functional.one_hot(actions)
    rewards = torch.tensor(np.array([sars.reward for sars in training_batch]), requires_grad=False)
    next_states = torch.tensor(np.array([sars.next_state for sars in training_batch]), requires_grad=False)
    fails = torch.tensor(np.array([sars.failed for sars in training_batch]), dtype=int, requires_grad=False)
    # Step 12
    next_action_probs = policy.policy_network.forward(next_states).detach()
    assert not next_action_probs.requires_grad
    next_actions_categorical = torch.distributions.Categorical(next_action_probs)
    next_actions = next_actions_categorical.sample()
    assert not next_actions.requires_grad
    next_actions_q_min = q_min(
        policy.q1_target_network,
        policy.q2_target_network,
        next_states,
        next_actions)
    assert not next_actions_q_min.requires_grad
    next_actions_log_probs = next_actions_categorical.log_prob(next_actions)
    assert not next_actions_log_probs.requires_grad
    y = rewards + GAMMA * (1-fails) * (next_actions_q_min - ALPHA * next_actions_log_probs)
    assert not y.requires_grad
    # Step 13
    for qi, q, opt in ((1, policy.q1_network, policy.q1_optimizer),
                       (2, policy.q2_network, policy.q2_optimizer)):
        assert not states.requires_grad
        assert not actions_hot.requires_grad
        q_state_action = torch.sum(q.forward(states) * actions_hot, 1)
        assert q_state_action.requires_grad
        q_loss = torch.mean((q_state_action - y)**2)
        stats[f'train/q_loss_{qi}'] = q_loss
        assert q_loss.requires_grad
        opt.zero_grad()
        q_loss.backward()
        opt.step()
    # Step 14
    action_probs = policy.policy_network.forward(states)
    assert action_probs.requires_grad
    actions_categorical = torch.distributions.Categorical(action_probs)
    actions = actions_categorical.sample()
    assert not actions.requires_grad
    actions_q_min = q_min(
        policy.q1_network,
        policy.q2_network,
        states,
        actions)
    assert not actions_q_min.requires_grad
    actions_log_probs = actions_categorical.log_prob(actions)
    assert actions_log_probs.requires_grad
    policy_loss = -1 * torch.mean(actions_q_min - ALPHA * actions_log_probs)
    stats['train/policy_loss'] = policy_loss
    policy.policy_optimizer.zero_grad()
    policy_loss.backward()
    policy.policy_optimizer.step()
    # Step 15
    polyak_update(policy.q1_target_network, policy.q1_network, tau=TAU)
    polyak_update(policy.q2_target_network, policy.q2_network, tau=TAU)
    return stats

In [43]:
tb_writer = SummaryWriter()

oss = env.observation_space.shape
if len(oss) != 1:
    raise RuntimeError(f'Unknown observation_space.shape: {oss}')
os_len = oss[0]
policy = Policy(os_len, env.action_space.n)

replay_buffer = deque(maxlen=10_000)

for episode in tqdm.tqdm(range(1, 1000+1)):
    episode_reward = run_episode(action, step, env, policy)
    tb_writer.add_scalar('main/episode_reward', episode_reward, episode)
    tb_writer.add_scalar('main/replay_buffer_length', len(replay_buffer), episode)
    for training_iteration in range(1, 100+1):
        stats = train(policy, replay_buffer)
        for stat, value in stats.items():
            tb_writer.add_scalar(stat, value, episode)

 38%|████████████████████████████████████████████▎                                                                      | 385/1000 [03:13<05:09,  1.99it/s]


KeyboardInterrupt: 